In [ ]:
import os
os.chdir('../../..')

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
from scipy.integrate import odeint
from sklearn.linear_model import Lasso
import pysindy as ps
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import interact
import ipywidgets as widgets

## Define three tank dynamic system model

I just took a random paper that wrote down the dynamic system equations for the three tank setup. Found [this](http://www.scs-europe.net/dlib/2016/ecms2016acceptedpapers/0347-mct_ECMS_0022.pdf).

![](../pics/three_tank_system_picture.png)

According to that paper, the system can be modelled like so:
$$
C\frac{dh_1(t)}{dt} = Q_1(t) - sign(h_1(t) - h_2(t)) \sqrt{|h_1(t) - h_2(t)|}\\
C\frac{dh_2(t)}{dt} = sign(h_1(t) - h_2(t)) \sqrt{|h_1(t)- h_2(t)|}\\
$$
Note that this is somewhat simpler than the equations in the paper, since we assume no leakage and tanks of equal sizes and equal valves...


In [ ]:
# Constants
A = 5
g = 9.81
C = np.sqrt(2*g)/A
q1 = 0
q3 = 0
C

In [ ]:
def system_dynamics_function(x, t):
    x1 = x[0]
    x2 = x[1]
    dh1_dt =C * q1 - C * np.sign(x1 - x2) * np.sqrt(np.abs(x1 - x2))
    dh2_dt = C * np.sign(x1 - x2) * np.sqrt(np.abs(x1 - x2)) 
    return dh1_dt, dh2_dt

In [ ]:
# define time steps
t = np.linspace(0, 10, 101)

In [ ]:
# initial condition
x0 = (1, 100)

In [ ]:
# solve ode
y = odeint(system_dynamics_function, x0, t)/100
h1 = y[:, 0]
h2 = y[:, 1]

In [ ]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
# signal 1
for sig, name in zip([h1, h2], ['h_1(t)', 'h_2(t)']):
    fig.add_trace(go.Scatter(x=t, y=sig, name=name,
                  mode="lines", opacity=1),
        row=1, col=1)
fig.update_xaxes(title_text='time')
fig.update_yaxes(title_text='x')
fig.update_layout(title_text="Fill levels over time", showlegend=True)
fig.show()

In [ ]:
library_functions = [
    lambda x : np.exp(x),
    lambda x : 1./x,
    lambda x : x,
    lambda x,y : np.sin(x+y),
    lambda x,y : np.sign(x-y)*np.sqrt(np.abs(x-y)),
    lambda x: x**2,
    lambda x: np.sqrt(x),
    lambda x,y: x*y
]
library_function_names = [
    lambda x : 'exp(' + x + ')',
    lambda x : '1/' + x,
    lambda x : x,
    lambda x,y : 'sin(' + x + ',' + y + ')',
    lambda x,y : 'sign('+x+'-'+y+')*sqrt('+x+' - '+y+')',
    lambda x: '{'+x+'}^2',
    lambda x: f'sqrt({x})',
    lambda x,y: f'{x}*{y}'
    
]
feature_library = ps.CustomLibrary(
    library_functions=library_functions, function_names=library_function_names
)

In [ ]:
2.0

In [ ]:
ps.feature_library.polynomial_library.PolynomialLibrary(degree=3).fit(y).get_feature_names()

In [ ]:
dt = .1
differentiation_method = ps.FiniteDifference(order=1)
optimizer = ps.STLSQ(threshold=0.04)
model = ps.SINDy(
    differentiation_method=differentiation_method,
    feature_library=feature_library,
    optimizer=optimizer,
    feature_names=["h1", "h2", "h3"]
)
model.fit(y, t=dt)
model.print()

In [ ]:
sim = model.simulate((.1, 1), t=t)

In [ ]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
# original samples
for sig, name in zip([h1, h2], ['h_1(t)', 'h_2(t)']):
    fig.add_trace(go.Scatter(x=t, y=sig, name=name,
                  mode="lines", opacity=1),
        row=1, col=1)
# model output
for sig, name in zip([sim[:, 0], sim[:, 1]], ['hm_1(t)', 'hm_2(t)']):
    fig.add_trace(go.Scatter(x=t, y=sig, name=name,
                  mode="lines", opacity=1),
        row=1, col=1)
fig.update_xaxes(title_text='time')
fig.update_yaxes(title_text='x')
fig.update_layout(title_text="...", showlegend=True)
fig.show()